In [56]:
import pandas as pd
import numpy as np
import joblib

In [108]:
rfg = joblib.load('random_forest.joblib')
df_profile = pd.read_csv("crude_profile.csv")

In [ ]:
def calculate_properties(name1,volume1,name2,volume2):
    profile1 = df_profile[df_profile['Name']==name1].iloc[0, :].to_dict()
    profile2 = df_profile[df_profile['Name']==name2].iloc[0, :].to_dict()

    # assume the volume of the mixture is the sum of the two crude volume.
    volume = volume1 + volume2
    v_portion1 = volume1 / volume
    v_portion2 = volume2 / volume
    mass = volume1 * profile1['Absolute Density (kg/m3)'] + volume2 * profile2['Absolute Density (kg/m3)']
    m_portion1 = volume1 * profile1['Absolute Density (kg/m3)'] / mass
    m_portion2 = volume2 * profile2['Absolute Density (kg/m3)'] / mass

    density = mass / volume
    gravity = v_portion1 * profile1['Gravity (&degAPI)'] + v_portion2 * profile2['Gravity (&degAPI)']
    sulphur =  m_portion1 * profile1['Sulphur (wt%)'] + m_portion2 * profile2['Sulphur (wt%)']
    mcr =  m_portion1 * profile1['MCR (wt%)'] + m_portion2 * profile2['MCR (wt%)']
    sediment = m_portion1 * profile1['Sediment (ppmw)'] + m_portion2 * profile2['Sediment (ppmw)']
    tan = m_portion1 * profile1['TAN (mgKOH/g)'] + m_portion2 * profile2['TAN (mgKOH/g)']
    salt = v_portion1 * profile1['Salt (ptb)'] + v_portion2 * profile2['Salt (ptb)']
    nickel = m_portion1 * profile1['Nickel (mg/kg)'] + m_portion2 * profile2['Nickel (mg/kg)']
    vanadium = m_portion1 * profile1['Vanadium (mg/kg)'] + m_portion2 * profile2['Vanadium (mg/kg)']
    c2 = v_portion1 * profile1['C2 Ethane (vol%)'] + v_portion2 * profile2['C2 Ethane (vol%)']
    c3 = v_portion1 * profile1['C3 Propane (vol%)'] + v_portion2 * profile2['C3 Propane (vol%)']
    c4i = v_portion1 * profile1['iC4 iso-Butane (vol%)'] + v_portion2 * profile2['iC4 iso-Butane (vol%)']
    c4n = v_portion1 * profile1['nC4 n-Butane (vol%)'] + v_portion2 * profile2['nC4 n-Butane (vol%)']
    c5i = v_portion1 * profile1['iC5 iso-Pentane (vol%)'] + v_portion2 * profile2['iC5 iso-Pentane (vol%)']
    c5n = v_portion1 * profile1['nC5 n-Pentane (vol%)'] + v_portion2 * profile2['nC5 n-Pentane (vol%)']
    c6 = v_portion1 * profile1['C6 Hexanes (vol%)'] + v_portion2 * profile2['C6 Hexanes (vol%)']
    c7 = v_portion1 * profile1['C7 Heptanes (vol%)'] + v_portion2 * profile2['C7 Heptanes (vol%)']
    c8 = v_portion1 * profile1['C8 Octanes (vol%)'] + v_portion2 * profile2['C8 Octanes (vol%)']
    c9 = v_portion1 * profile1['C9 Nonanes (vol%)'] + v_portion2 * profile2['C9 Nonanes (vol%)']
    c10 = v_portion1 * profile1['C9 Nonanes (vol%)'] + v_portion2 * profile2['C9 Nonanes (vol%)']

    x_pred = [density, gravity, sulphur, mcr, sediment, tan, salt, nickel, vanadium,
             c2, c3, c4i, c4n, c5i, c5n, c6, c7, c8, c9, c10]

    df_x_pred = pd.DataFrame(np.array(x_pred*21).reshape(21,-1))
    df_x_pred.columns = df_profile.columns[:-1]

    mass_list = list(range(0,100,5))
    mass_list.append(99)
    df_x_pred['%mass'] = np.array(mass_list).reshape(21,1)

    rfg.predict(df_x_pred)

In [37]:
name1 = "Pembina"
name2 = "Fort Hills Dilbit"
volume1 = 1
volume2 = 1

In [32]:
profile1 = df_profile[df_profile['Name']==name1].iloc[0, :].to_dict()
profile2 = df_profile[df_profile['Name']==name2].iloc[0, :].to_dict()

In [43]:
# assume the volume of the mixture is the sum of the two crude volume.
volume = volume1 + volume2
v_portion1 = volume1 / volume
v_portion2 = volume2 / volume
mass = volume1 * profile1['Absolute Density (kg/m3)'] + volume2 * profile2['Absolute Density (kg/m3)']
m_portion1 = volume1 * profile1['Absolute Density (kg/m3)'] / mass
m_portion2 = volume2 * profile2['Absolute Density (kg/m3)'] / mass

density = mass / volume
gravity = v_portion1 * profile1['Gravity (&degAPI)'] + v_portion2 * profile2['Gravity (&degAPI)']
sulphur =  m_portion1 * profile1['Sulphur (wt%)'] + m_portion2 * profile2['Sulphur (wt%)']
mcr =  m_portion1 * profile1['MCR (wt%)'] + m_portion2 * profile2['MCR (wt%)']
sediment = m_portion1 * profile1['Sediment (ppmw)'] + m_portion2 * profile2['Sediment (ppmw)']
tan = m_portion1 * profile1['TAN (mgKOH/g)'] + m_portion2 * profile2['TAN (mgKOH/g)']
salt = v_portion1 * profile1['Salt (ptb)'] + v_portion2 * profile2['Salt (ptb)']
nickel = m_portion1 * profile1['Nickel (mg/kg)'] + m_portion2 * profile2['Nickel (mg/kg)']
vanadium = m_portion1 * profile1['Vanadium (mg/kg)'] + m_portion2 * profile2['Vanadium (mg/kg)']
c2 = v_portion1 * profile1['C2 Ethane (vol%)'] + v_portion2 * profile2['C2 Ethane (vol%)']
c3 = v_portion1 * profile1['C3 Propane (vol%)'] + v_portion2 * profile2['C3 Propane (vol%)']
c4i = v_portion1 * profile1['iC4 iso-Butane (vol%)'] + v_portion2 * profile2['iC4 iso-Butane (vol%)']
c4n = v_portion1 * profile1['nC4 n-Butane (vol%)'] + v_portion2 * profile2['nC4 n-Butane (vol%)']
c5i = v_portion1 * profile1['iC5 iso-Pentane (vol%)'] + v_portion2 * profile2['iC5 iso-Pentane (vol%)']
c5n = v_portion1 * profile1['nC5 n-Pentane (vol%)'] + v_portion2 * profile2['nC5 n-Pentane (vol%)']
c6 = v_portion1 * profile1['C6 Hexanes (vol%)'] + v_portion2 * profile2['C6 Hexanes (vol%)']
c7 = v_portion1 * profile1['C7 Heptanes (vol%)'] + v_portion2 * profile2['C7 Heptanes (vol%)']
c8 = v_portion1 * profile1['C8 Octanes (vol%)'] + v_portion2 * profile2['C8 Octanes (vol%)']
c9 = v_portion1 * profile1['C9 Nonanes (vol%)'] + v_portion2 * profile2['C9 Nonanes (vol%)']
c10 = v_portion1 * profile1['C9 Nonanes (vol%)'] + v_portion2 * profile2['C9 Nonanes (vol%)']

x_pred = [density, gravity, sulphur, mcr, sediment, tan, salt, nickel, vanadium,
         c2, c3, c4i, c4n, c5i, c5n, c6, c7, c8, c9, c10]

In [106]:
# np.hstack((np.array(x_pred*21).reshape(21,-1), np.array(mass_list).reshape(21,1)))

In [85]:
df_x_pred = pd.DataFrame(np.array(x_pred*21).reshape(21,-1))
df_x_pred.columns = df_profile.columns[:-1]

mass_list = list(range(0,100,5))
mass_list.append(99)
df_x_pred['%mass'] = np.array(mass_list).reshape(21,1)

In [109]:
rfg.predict(df_x_pred)

array([ 31.242321  ,  43.29641341,  92.11234904, 118.04084877,
       153.23736424, 176.72090877, 214.3970963 , 248.90133438,
       276.55778384, 332.38854111, 366.21829764, 411.118121  ,
       428.1001544 , 457.2191745 , 491.24149158, 532.11725274,
       588.43242043, 637.32417985, 688.63543744, 687.08548103,
       687.88266866])

In [ ]:
http://127.0.0.1:8080/?name1=Pembina&volume1=1&name2=Cold_Lake&volume2=1

In [35]:
profile1

{'Absolute Density (kg/m3)': 825.1,
 'Gravity (&degAPI)': 39.9,
 'Sulphur (wt%)': 0.41,
 'MCR (wt%)': 1.9,
 'Sediment (ppmw)': 0.0,
 'TAN (mgKOH/g)': 0.0,
 'Salt (ptb)': 0.0,
 'Nickel (mg/kg)': 4.5,
 'Vanadium (mg/kg)': 9.6,
 'C2 Ethane (vol%)': 0.02,
 'C3 Propane (vol%)': 0.42,
 'iC4 iso-Butane (vol%)': 1.03,
 'nC4 n-Butane (vol%)': 2.62,
 'iC5 iso-Pentane (vol%)': 1.4,
 'nC5 n-Pentane (vol%)': 1.82,
 'C6 Hexanes (vol%)': 5.85,
 'C7 Heptanes (vol%)': 7.18,
 'C8 Octanes (vol%)': 7.0,
 'C9 Nonanes (vol%)': 6.29,
 'C10 Decanes (vol%)': 3.58,
 'Name': 'Pembina'}